### Generate some photometry files to fit from a table
This is an example of how to generate some photometry files that can be read in and fit with `sdf`. In this case each row is a single target that has photometry from mulitple catalogues.

A hash and temporary file is used so that files are only updated if they have changed. This is convenient if only some of the files change for some reason and the fitting has already been run (which only updates if the photometry file is more recent than the fit).

In [1]:
import os
import hashlib
import numpy as np
from astropy.table import Table

In [2]:
t = Table.read('path/to/selection_photo339.csv')
t.columns

<TableColumns names=('Source','RA_ICRS','DE_ICRS','e_RAdeg','e_DEdeg','Plx','e_Plx','RPlx','PM','pmRA','e_pmRA','pmDE','e_pmDE','RUWE','FG','e_FG','Gmag','e_Gmag','o_BPmag','FBP','e_FBP','RFBP','BPmag','e_BPmag','FRP','e_FRP','RFRP','RPmag','e_RPmag','E(BP/RP)','BP-RP','BP-G','G-RP','RV','e_RV','GRVSmag','e_GRVSmag','o_GRVSmag','Teff','b_Teff_x','B_Teff_xa','logg','b_logg_x','B_logg_xa','[Fe/H]','b_[Fe/H]_x','B_[Fe/H]_xa','Dist','b_Dist_x','B_Dist_xa','A0','b_A0_x','B_A0_xa','AG','b_AG_x','B_AG_xa','E(BP-RP)','b_E(BP-RP)_x','B_E(BP-RP)_xa','Lib','BTmag','e_BTmag','VTmag','e_VTmag','Bmag','Vmag','Rmag','AllWISE','RAJ2000','DEJ2000','eeMaj','eeMin','eePA','W1mag','W2mag','W3mag','W4mag','Jmag','Hmag','Kmag','e_W1mag','e_W2mag','e_W3mag','e_W4mag','e_Jmag','e_Hmag','e_Kmag','ID','ccf','ex','var','qph')>

In [3]:
# as a cheat read in an existing photometry file to use as a template
o = Table.read('/Users/grant/tmp/sdb/masters/sdb-v2-011125.42+152621.5/public/sdb-v2-011125.42+152621.5-rawphot.txt', format='ipac')
o.remove_rows(np.arange(len(o)))
o.meta['comments'].pop()
o

Band,Phot,Err,Sys,Lim,Unit,bibcode,Note1,Note2,SourceID,private,exclude
str7,float64,float64,float64,int64,str3,str19,str14,str10,str25,int64,int64


In [4]:
def filehash(file):
    """Return an md5 hash for a given file, ignoring IPAC comments."""
    hasher = hashlib.md5()
    buf = b''
    with open(file,'r') as f:
        for text in f:
            if len(text) <= 2: # "\" and newline at a minimum
                pass
            elif text[0:2] == '\ ':
                pass
            else:
                #                print(len(text),"|"+text+"|")
                buf += text.encode()

    hasher.update(buf)
    return hasher.hexdigest()

In [5]:
for r in t:
    phot = o.copy()

    # meta
    phot.meta['keywords']['main_id'] = {'value': f'Gaia DR3 {r["Source"]}'}
    phot.meta['keywords']['raj2000'] = {'value': r['RA_ICRS']}
    phot.meta['keywords']['dej2000'] = {'value': r['DE_ICRS']}
    phot.meta['keywords']['sp_type'] = {'value': None}
    phot.meta['keywords']['sp_bibcode'] = {'value': None}
    phot.meta['keywords']['plx_value'] = {'value': r['Plx']}
    phot.meta['keywords']['plx_err'] = {'value': r['e_Plx']}
    phot.meta['keywords']['plx_bibcode'] = {'value': 'Gaia'}
    phot.meta['keywords']['id'] = phot.meta['keywords']['main_id']

    # photometry
    phot.add_row(['BT', r['BTmag'], r['e_BTmag'], 0.02, 0, 'mag', 'Tycho2', '', '', '', 0, 0])
    phot.add_row(['VT', r['VTmag'], r['e_VTmag'], 0.02, 0, 'mag', 'Tycho2', '', '', '', 0, 0])
    phot.add_row(['GAIA.BP', r['BPmag'], r['e_BPmag'], 0.02, 0, 'mag', 'Gaia', '', '', '', 0, 0])
    phot.add_row(['GAIA.RP', r['RPmag'], r['e_RPmag'], 0.02, 0, 'mag', 'Gaia', '', '', '', 0, 0])
    phot.add_row(['GAIA.G', r['Gmag'], r['e_Gmag'], 0.02, 0, 'mag', 'Gaia', '', '', '', 0, 0])
    phot.add_row(['2MJ', r['Jmag'], r['e_Jmag'], 0.02, 0, 'mag', '2MASS', '', '', '', 0, 0])
    phot.add_row(['2MH', r['Hmag'], r['e_Hmag'], 0.02, 0, 'mag', '2MASS', '', '', '', 0, 0])
    phot.add_row(['2MKS', r['Kmag'], r['e_Kmag'], 0.02, 0, 'mag', '2MASS', '', '', '', 0, 0])
    phot.add_row(['WISE3P4', r['W1mag'], r['e_W1mag'], 0.02, int(r['qph'][0] == 'U'), 'mag', 'WISE', '', '', '', 0, int(r['ccf'][0] in ['H', 'D', 'P', 'O'])])
    phot.add_row(['WISE4P6', r['W2mag'], r['e_W2mag'], 0.02, int(r['qph'][1] == 'U'), 'mag', 'WISE', '', '', '', 0, int(r['ccf'][1] in ['H', 'D', 'P', 'O'])])
    phot.add_row(['WISE12', r['W3mag'], r['e_W3mag'], 0.02, int(r['qph'][2] == 'U'), 'mag', 'WISE', '', '', '', 0, int(r['ccf'][2] in ['H', 'D', 'P', 'O'])])
    phot.add_row(['WISE22', r['W4mag'], r['e_W4mag'], 0.02, int(r['qph'][3] == 'U'), 'mag', 'WISE', '', '', '', 0, int(r['ccf'][3] in ['H', 'D', 'P', 'O'])])

    wp = f'/Users/grant/tmp/sdb/masters/herbig/{r["Source"]}'
    file = f'{wp}/{r["Source"]}-rawphot.txt'
    if not os.path.exists(wp):
        os.mkdir(wp)

    oldhash = filehash(file)
    phot.write('/tmp/tmp.txt', format='ipac', overwrite=True)
    newhash = filehash('/tmp/tmp.txt')
    if oldhash != newhash:
        os.rename('/tmp/tmp.txt', file)
        print(f'updating {file}')
    else:
        # print(f'not updating {file}')
        pass

    if os.path.exists('/tmp/tmp.txt'):
        os.remove('/tmp/tmp.txt')